# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui
sinasc.isnull().sum().sort_values(ascending=False).head(10)

DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
CODOCUPMAE     2907
MESPRENAT      2867
QTDFILMORT     2098
QTDPARTNOR     1879
QTDPARTCES     1791
dtype: int64

In [3]:
sinasc.isna().sum().sort_values(ascending=False).head(10)

DTRECORIGA    27028
CODANOMAL     26814
IDADEPAI      19421
SERIESCMAE    12710
DTULTMENST    10072
CODOCUPMAE     2907
MESPRENAT      2867
QTDFILMORT     2098
QTDPARTNOR     1879
QTDPARTCES     1791
dtype: int64

In [4]:
# 3) seu código aqui
selecao = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
for i in selecao:
    print (str(i)+('  ')+str(sinasc[i].isna().sum()))

LOCNASC  0
IDADEMAE  0
ESTCIVMAE  317
ESCMAE  312
QTDFILVIVO  1573
GESTACAO  1232
GRAVIDEZ  79
CONSULTAS  0
APGAR5  103


In [5]:
df_select = pd.DataFrame()

In [6]:
df_select = sinasc[selecao] 
df_select

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [7]:
df_select.isna().sum().sort_values(ascending=False).head(10)

QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
CONSULTAS        0
IDADEMAE         0
LOCNASC          0
dtype: int64

In [8]:
# 4) seu código aqui
print(df_select.shape)
df_select.dropna(subset=['APGAR5'],inplace=True) 
print(df_select.shape)

(27028, 9)
(26925, 9)


C:\Users\felip\AppData\Local\Temp\ipykernel_9872\2750372090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select.dropna(subset=['APGAR5'],inplace=True)


In [9]:
import numpy as np

In [10]:
#observe que as variáveis possuem o código ,
#que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código .['ESTCIVMAE', 'CONSULTAS']99
# 5) seu código aqui
df_select['ESTCIVMAE'].replace(9, np.nan,inplace=True)
df_select['CONSULTAS'].replace(9, np.nan,inplace=True)




C:\Users\felip\anaconda3\lib\site-packages\pandas\core\series.py:4576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [11]:
print(df_select.shape)
df_select.dropna(subset=['ESTCIVMAE','CONSULTAS'],inplace=True)
print(df_select.shape)

(26925, 9)
(26416, 9)


C:\Users\felip\AppData\Local\Temp\ipykernel_9872\2292478035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select.dropna(subset=['ESTCIVMAE','CONSULTAS'],inplace=True)


In [12]:
# 6) Seu código aqui
# Substitua os valores faltantes da quantitativa () por zero. QTDFILVIVO
df_select['QTDFILVIVO'].fillna(0,inplace=True)
df_select.QTDFILVIVO.isna().sum()


C:\Users\felip\anaconda3\lib\site-packages\pandas\core\series.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


0

In [13]:
# 7) seu código aqui
df_select.isna().sum().sort_values(ascending=False)

GESTACAO      1174
ESCMAE         242
GRAVIDEZ        73
APGAR5           0
CONSULTAS        0
QTDFILVIVO       0
ESTCIVMAE        0
IDADEMAE         0
LOCNASC          0
dtype: int64

In [14]:
df_select['GESTACAO'].fillna(method='ffill',inplace=True)
df_select.GESTACAO.isna().sum()

0

In [15]:
df_select['ESCMAE'].fillna(method='ffill',inplace=True)
df_select.ESCMAE.isna().sum()

0

In [16]:
df_select['GRAVIDEZ'].fillna(method='bfill',inplace=True)
df_select.GRAVIDEZ.isna().sum()

0

In [17]:
df_select.isna().sum().sort_values(ascending=False)

APGAR5        0
CONSULTAS     0
GRAVIDEZ      0
GESTACAO      0
QTDFILVIVO    0
ESCMAE        0
ESTCIVMAE     0
IDADEMAE      0
LOCNASC       0
dtype: int64

In [18]:
#Entre 8 e 10 está em uma faixa 'normal'.
#Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
#Entre 4 e 5 significa 'asfixia moderada'.
#Entre 0 e 3 significa 'asfixia severa'.

In [19]:
# 8) seu código aqui
df_select.loc[df_select['APGAR5']>=8, 'APGAR5_CAT'] = 'normal'
df_select.loc[(df_select['APGAR5']>=6) & (df_select['APGAR5']<=7 ) , 'APGAR5_CAT'] = 'asfixia leve'
df_select.loc[(df_select['APGAR5']>=4) & (df_select['APGAR5']<=5 ) , 'APGAR5_CAT'] = 'asfixia moderada'
df_select.loc[df_select['APGAR5']<=3, 'APGAR5_CAT'] = 'asfixia severa'



C:\Users\felip\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\felip\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\felip\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [20]:
# 9) seu código aqui
df_select.columns = df_select.columns.str.lower()

In [21]:
df_select.columns

Index(['locnasc', 'idademae', 'estcivmae', 'escmae', 'qtdfilvivo', 'gestacao',
       'gravidez', 'consultas', 'apgar5', 'apgar5_cat'],
      dtype='object')